In [1]:
from os import stat
import pandas as pd
from sqlalchemy.sql.elements import Label
from sqlalchemy.sql.functions import user
from doc_anno_suite.database import engine
from doc_anno_suite.database import db_session
from doc_anno_suite.models import User, Image, Schema, Annotation
from sqlalchemy import text
from datetime import datetime
from dataclasses import asdict
from doc_anno_suite.annotations.dbutils import DatabaseCRUD
import time

In [44]:
docanno2visanno = {
    'lab_1':'T50',
    'lab_2':'T59',
    'lab_3':'T56',
    'lab_4':'T58',
    'lab_5':'T51',
    'lab_6':'T57',
    'lab_7':'T54',
    'lab_8':'T53',
    'lab_9':'T52',
    'lab_10':'T33',
    'lab_11':'T63',
    'lab_12':'T60',
    'lab_13':'T73',
    'lab_14':'T65',
    'lab_15':'tagText',
    'lab_17':'T26',
    'lab_18':'T27',
    'lab_19':'T28',
    'lab_21':'T70',
    'lab_22':'T71',
    'lab_23':'T69',
    'lab_24':'T72',
    'lab_25':'3',
    'lab_26':'nest-0',
    'lab_27':'nest-1',
    'lab_28':'nest-2',
    'lab_29':'3',
    'lab_30':'hardness-1',
    'lab_31':'hardness-2',
    'lab_32':'hardness-3',
    'lab_34':'commentText',
}

iconsUrlDic = {
    'T50': '../images/icons/bar-0.png',
    'T51': '../images/icons/area-0.png',
    'T52': '../images/icons/flowchart-0.png',
    'T53': '../images/icons/glyph-0.png',
    'T54': '../images/icons/matrix-0.png',
    'T55': '../images/icons/illustrative-0.png',
    'T56': '../images/icons/line-0.png',
    'T57': '../images/icons/surface-0.png',
    'T58': '../images/icons/nodelinkcir-0.png',
    'T59': '../images/icons/scatter-0.png',
    'T60': '../images/icons/text-0.png',
    'T61': '../images/icons/treemap-0.png',
    'T62': '../images/icons/volume-0.png',
    'T63': '../images/icons/continuous-0.png',
    'T73': '../images/icons/color-0.png',
    'T64': '../images/icons/no-0.png',
    'T65': '../images/icons/cannottell-0.png',
    'T66': '../images/icons/other-0.png',
    'T1': '../images/icons/bar-0.png',
    'T2': '../images/icons/map-0.png',
    'T3': '../images/icons/nodelinktree-0.png',
    'T4': '../images/icons/flowchart-0.png',
    'T5': '../images/icons/flowline-0.png',
    'T6': '../images/icons/glyph-0.png',
    'T7': '../images/icons/heatmap-0.png',
    'T8': '../images/icons/illustrative-0.png',
    'T9': '../images/icons/surface-0.png',
    'T10': '../images/icons/line-0.png',
    'T11': '../images/icons/matrix-0.png',
    'T12': '../images/icons/nodelinkcir-0.png',
    'T13': '../images/icons/parallel-0.png',
    'T14': '../images/icons/pie-0.png',
    'T15': '../images/icons/pointcloud-0.png',
    'T16': '../images/icons/scatter-0.png',
    'T17': '../images/icons/tag-0.png',
    'T18': '../images/icons/timeline-0.png',
    'T19': '../images/icons/treemap-0.png',
    'T20': '../images/icons/volume-0.png',
    'T21': '../images/icons/voronoi-0.png',
    'T22': '../images/icons/mesh-0.png',
    'T23': '../images/icons/no-0.png',
    'T24': '../images/icons/cannottell-0.png',
    'T25': '../images/icons/other-0.png',
    'T26': '../images/icons/2D-0.png',
    'T27': '../images/icons/3D-0.png',
    'T28': '../images/icons/cannottell-0.png',
    'T29': '../images/icons/photo-0.png',
    'T30': '../images/icons/tech-0.png',
    'T31': '../images/icons/quan-0.png',
    'T32': '../images/icons/flowchart-0.png',
    'T33': '../images/icons/GUI-0.png',
    'T34': '../images/icons/cannottell-0.png',
    'T35': '../images/icons/other-0.png',
    'T36': '../images/icons/topology-0.png',
    'T37': '../images/icons/displot-0.png',
    'T38': '../images/icons/inte-surface-0.png',
    'T39': '../images/icons/mesh-surface-0.png',
    'type_unsure': '../images/icons/unsure-0.png',
    'dim_unsure': '../images/icons/unsure-0.png',
    'function_unsure': '../images/icons/unsure-0.png',
    'T70': '../images/icons/yes-manu-0.png',
    'T71': '../images/icons/yes-mach-0.png',
    'T69': '../images/icons/one-vis-0.png',
    'T72': '../images/icons/cannottell-0.png',
    'nest-0': '../images/icons/comp-n-0.png',
    'nest-1': '../images/icons/comp-y-0.png',
    'nest-2': '../images/icons/maybe-0.png',
    'hardness-1': '../images/icons/easy-0.png',
    'hardness-2': '../images/icons/neutral-0.png',
    'hardness-3': '../images/icons/hard-0.png'
}

In [4]:
schema_data = pd.read_csv('dummy data/label_schema.csv', encoding = 'utf-8')

In [17]:
for i in range(len(schema_data)):
    if(schema_data.at[i,'M_label_type'] != 0):
        label_id = schema_data.at[i,'M_label_id']
        visanno_id = docanno2visanno[label_id]
        if(visanno_id in iconsUrlDic):
            icon_url = iconsUrlDic[visanno_id]
            icon_url = '../static/images/customized_icons/' + icon_url.split('/')[-1]
            
        else:
            icon_url = ''
        schema_data.at[i,'O_label_icon'] = icon_url
schema_data.to_csv('dummy data/label_schema_new.csv', index=False, encoding='utf_8_sig')

In [15]:
a = [12,3,4]
a[-1]

4

In [23]:
user_data = pd.read_csv('dummy data/userTableFormalNew.csv', encoding = 'utf-8')
user_data.head(1)

,userID,userIndex,currentIndex,userName,userEmail,userInstitution,userPI,userPIEmail,userOthers,userReason,userImages,facultyCoder,userImagesV2,cocodeSequence,cocodeIndex,cocodeSequenceV2,cocodeIndexV2,history_logs,history_list_date
0,MS,1,1462,Michael Sedlmair,1,1,1,1,1,1,I237Y1995; I263Y2000; I1022Y2005; I1061Y2005; ...,1,I4Y2005; I92Y1990; I101Y2000; I382Y1990; I8005...,I895Y2005; I122Y2000; I370Y2000; I467Y1995; I3...,49,I274Y2010; I7661Y2020; I1085Y2015; I634Y2000; ...,49,"Log:STARTW;MS:{""authors"":""MS,PI"",""tags"":"""",""an...","placeHolder;MS-9/24/2021, 10:09:10;MS-9/24/202..."


In [25]:
JC_assignment = ''
for i in range(len(user_data)):
    if(user_data.at[i,'userID'] == 'JC'):
        JC_assignment = user_data.at[i,'userImagesV2'].replace('; ',';')
        JC_arr = JC_assignment.split(';')
        print(len(JC_arr))

3177


In [28]:
users = pd.read_csv('dummy data/vis7years/users.csv', encoding = 'utf-8')
for i in range(len(users)):
    if(users.at[i,'M_username'] == 'JC'):
        users.at[i,'M_assignment_by_image_id'] = JC_assignment 

In [29]:
users.to_csv('dummy data/vis7years/user_new.csv', index=False, encoding='utf_8_sig')

In [30]:
class Utils():

    @staticmethod
    def createTableDicByData(dicID, data):
        dataDic = {}
        column_list = data.columns.values.tolist()
        for i in range(len(data)):
            dataID = str(data.loc[i,dicID])
            if(dataID in dataDic):
                dataInfo = {}
                for key in column_list:
                    dataInfo[key] = data.loc[i,key]
                dataDic[dataID].append(dataInfo)
            else:
                dataInfo = {}
                for key in column_list:
                    dataInfo[key] = data.loc[i,key]
                dataDic[dataID] = [dataInfo]
        return dataDic 

    @staticmethod
    def createTableDicTwoByData(dicID1, dicID2, data):
        dataDic = {}
        column_list = data.columns.values.tolist()
        for i in range(len(data)):
            dataID1 = data.loc[i,dicID1]
            dataID2 = data.loc[i,dicID2]
            dataID = str(dataID1) + '-' + str(dataID2)
            if(dataID in dataDic):
                dataInfo = {}
                for key in column_list:
                    dataInfo[key] = data.loc[i,key]
                dataDic[dataID].append(dataInfo)
            else:
                dataInfo = {}
                for key in column_list:
                    dataInfo[key] = data.loc[i,key]
                dataDic[dataID] = [dataInfo]
        return dataDic 

In [47]:
anno_data_old = pd.read_csv('dummy data/annotation_table_V2.csv', encoding = 'utf-8')
anno_dic = Utils.createTableDicTwoByData('imageID', 'userID', anno_data_old)

In [48]:
data = pd.read_csv('dummy data/raw_annotations.csv', encoding = 'utf-8')
for i in range(len(data)):
    image_id = data.at[i,'image_id']
    username = data.at[i,'username']
    key = image_id + '-' + username
    record = anno_dic[key][0]
    data.at[i,'is_error_image'] = int(record['isMark'])
    data.at[i,'need_discuss'] = int(record['needAnotherCoder'])
    data.at[i,'marked_fun'] = int(record['isFunFigure'])
    data.at[i,'marked_OK'] = int(record['isHappyFigure'])
    for label_id in docanno2visanno:
        tag = docanno2visanno[label_id]
        if(tag in record):
            data.loc[i,label_id] = record[tag]
        else:
            if(label_id == 'lab_25'):
                if(record['ifNested'] == 0):
                    data.at[i,label_id] = 'lab_26'
                elif(record['ifNested'] == 1):
                    data.at[i,label_id] = 'lab_27'
                elif(record['ifNested'] == 2):
                    data.at[i,label_id] = 'lab_28'
            elif(label_id == 'lab_29'):
                if(record['hardness'] == 1):
                    data.at[i,label_id] = 'lab_30'
                elif(record['hardness'] == 2):
                    data.at[i,label_id] = 'lab_31'
                elif(record['hardness'] == 3):
                    data.at[i,label_id] = 'lab_32'

In [49]:
data.to_csv('dummy data/vis7years/raw_annotations_new.csv', index=False, encoding='utf_8_sig')